In [ ]:
#mediapipe 설치

!pip install mediapipe opencv-python
!apt-get update -qq >/dev/null
!apt-get install -y fonts-nanum

W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following NEW packages will be installed:
  fonts-nanum
0 upgraded, 1 newly installed, 0 to remove and 39 not upgraded.
Need to get 10.3 MB of archives.
After this operation, 34.1 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/universe amd64 fonts-nanum all 20200506-1 [10.3 MB]
Fetched 10.3 MB in 3s (3,861 kB/s)
Selecting previously unselected package fonts-nanum.
(Reading database ... 126675 files and directories currently installed.)
Preparing to unpack .../fonts-nanum_20200506-1_all.deb ...
Unpacking fonts-nanum (20200506-1) ...
Setting up fonts-nanum (20200506-1) ...
Processing triggers for fontconfig (2.13.1-4.2ubuntu5) ...


In [ ]:
# ------------------------------
# Cell 1: 기본 임포트 & Mediapipe 준비
# ------------------------------
from google.colab import output
from IPython.display import Javascript, display, clear_output
from base64 import b64decode
import cv2, mediapipe as mp, numpy as np, time

# Mediapipe 모듈 참조
mp_drawing = mp.solutions.drawing_utils
mp_pose = mp.solutions.pose

# 각도 계산 함수 (b를 중심으로 a-b-c 각도)
def calculate_angle(a, b, c):
    a, b, c = np.array(a), np.array(b), np.array(c)
    radians = np.arctan2(c[1]-b[1], c[0]-b[0]) - np.arctan2(a[1]-b[1], a[0]-b[0])
    angle = np.abs(radians * 180.0 / np.pi)
    if angle > 180.0:
        angle = 360 - angle
    return angle

print("✅ 준비 완료: JS+Canvas 캡처 방식 / Mediapipe Pose 사용")


✅ 준비 완료: JS+Canvas 캡처 방식 / Mediapipe Pose 사용


In [ ]:
# ------------------------------
# Cell 2: 브라우저 카메라 시작 (허용 팝업)
# ------------------------------
display(Javascript('''
  async function startWebcam() {
    // 이미 생성된 비디오가 있으면 재사용
    if (window.__colabVideo && !window.__colabVideo.paused) {
      return;
    }
    const video = document.createElement('video');
    video.style.display = 'block';
    video.style.maxWidth = '100%';
    document.body.appendChild(video);
    const stream = await navigator.mediaDevices.getUserMedia({ video: { facingMode: "user" }, audio: false });
    video.srcObject = stream;
    await video.play();
    // 전역 보관 (다른 셀에서 프레임 캡처에 사용)
    window.__colabVideo = video;
  }
  startWebcam();
'''))
print("📸 카메라 요청 완료 — 브라우저 팝업에서 '허용'을 눌러주세요.")


<IPython.core.display.Javascript object>

📸 카메라 요청 완료 — 브라우저 팝업에서 '허용'을 눌러주세요.


In [ ]:
# ------------------------------
# Cell 3: 프레임 캡처 함수 & 유틸
# ------------------------------
def get_colab_frame(jpeg_quality=0.8):
    """
    브라우저의 <video>에서 현재 프레임을 Canvas로 복사 → dataURL(JPEG) → 파이썬으로 전달
    """
    data_url = output.eval_js(f'''
      (async () => {{
        const video = window.__colabVideo;
        if (!video) return null;
        const canvas = document.createElement('canvas');
        canvas.width = video.videoWidth || 640;
        canvas.height = video.videoHeight || 480;
        const ctx = canvas.getContext('2d');
        ctx.drawImage(video, 0, 0);
        return canvas.toDataURL('image/jpeg', {jpeg_quality});
      }})()
    ''')
    if not data_url:
        return None
    img_bytes = b64decode(data_url.split(',')[1])
    frame = cv2.imdecode(np.frombuffer(img_bytes, np.uint8), cv2.IMREAD_COLOR)
    return frame

def draw_hud(vis, angle=None, count=0, fps=None, status_text=None):
    """
    결과 HUD 오버레이
    """
    y = 28
    if angle is not None:
        cv2.putText(vis, f"Angle: {int(angle)} deg", (18, y),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255,255,255), 2)
        y += 32
    cv2.putText(vis, f"Count: {count}", (18, y),
                cv2.FONT_HERSHEY_SIMPLEX, 0.85, (0,255,0), 2)
    y += 32
    if fps is not None:
        cv2.putText(vis, f"FPS: {fps:.1f}", (18, y),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255,255,0), 2)
        y += 28
    if status_text:
        cv2.putText(vis, status_text, (18, y),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0,200,255), 2)


In [ ]:
# ------------------------------
# Cell 4: 자동 캡처 + Mediapipe 분석 루프 (스쿼트)
# ------------------------------
from google.colab.patches import cv2_imshow

# Mediapipe Pose 초기화
pose = mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5)

counter, stage = 0, None
ema_fps, t_last = None, time.time()

print("✅ 자동 분석 시작 — 셀 정지(Stop) 버튼으로 종료하세요.")

try:
    while True:
        frame = get_colab_frame(jpeg_quality=0.8)
        if frame is None:
            # 아직 허용 전이거나 준비 중인 경우가 있을 수 있음
            time.sleep(0.05)
            continue

        # 원본 프레임을 복사해 그 위에 그립니다
        vis = frame.copy()

        # Mediapipe 추론 (RGB)
        img_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        results = pose.process(img_rgb)

        angle = None
        status_text = None

        if results.pose_landmarks:
            # 랜드마크 그리기
            mp_drawing.draw_landmarks(vis, results.pose_landmarks, mp_pose.POSE_CONNECTIONS)

            lm = results.pose_landmarks.landmark
            # 왼쪽 다리 기준: 엉덩이-무릎-발목
            hip   = [lm[mp_pose.PoseLandmark.LEFT_HIP.value].x,   lm[mp_pose.PoseLandmark.LEFT_HIP.value].y]
            knee  = [lm[mp_pose.PoseLandmark.LEFT_KNEE.value].x,  lm[mp_pose.PoseLandmark.LEFT_KNEE.value].y]
            ankle = [lm[mp_pose.PoseLandmark.LEFT_ANKLE.value].x, lm[mp_pose.PoseLandmark.LEFT_ANKLE.value].y]

            # 픽셀 좌표로 바꾸지 않아도 각도 비교에는 무방 (정규화 좌표)
            angle = calculate_angle(hip, knee, ankle)

            # 간단한 스쿼트 상태 전이: up(펴짐) -> down(앉음)
            if angle > 160:
                stage = "up"
                status_text = "Standing (UP)"
            if angle < 90 and stage == "up":
                stage = "down"
                counter += 1
                status_text = "Squat! (DOWN)"
        else:
            status_text = "No Pose Detected"

        # FPS 계산 (EMA)
        now = time.time()
        dt = now - t_last
        t_last = now
        inst_fps = 1.0 / dt if dt > 0 else 0.0
        ema_fps = inst_fps if ema_fps is None else (0.9 * ema_fps + 0.1 * inst_fps)

        # HUD 오버레이
        draw_hud(vis, angle=angle, count=counter, fps=ema_fps, status_text=status_text)

        # 출력 깜빡임을 줄이기 위해 clear_output(wait=True)
        clear_output(wait=True)
        cv2_imshow(vis)

        # 너무 과도한 요청 방지 (CPU/GPU/브라우저 부담 완화)
        time.sleep(0.06)  # 약 ~16 FPS 목표

except KeyboardInterrupt:
    print("🛑 사용자 중단")
finally:
    pose.close()
    print("💾 최종 스쿼트 횟수:", counter)


✅ 자동 분석 시작 — 셀 정지(Stop) 버튼으로 종료하세요.


In [ ]:
# ------------------------------
# Cell 5: 스트림 정리(선택) & 요약
# ------------------------------
# 브라우저 쪽 카메라 스트림 중지
display(Javascript('''
  (function stopWebcam() {
    const video = window.__colabVideo;
    if (video && video.srcObject) {
      for (const track of video.srcObject.getTracks()) {
        track.stop();
      }
      video.srcObject = null;
      video.remove();
    }
    window.__colabVideo = null;
  })();
'''))

print("""정리 완료 ✅
- 브라우저 비디오 스트림 중지
- Colab 셀(분석 루프)은 Stop으로 중단하세요
- 다시 시작하려면: 셀 2(카메라 시작) → 셀 4(분석 루프)
""")
